## Import all the needed packages

In [1]:
%matplotlib inline

# DataFrame
import pandas as pd

# Elasticsearch
from elasticsearch import Elasticsearch, helpers

# datetime
from datetime import datetime

# isnan()
import math

# plot
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

## This time, let's find out all the src/dest site names and IP addresses pairs

## Create the metadata sets

In [6]:
metadata_delay = set()
metadata_packetloss = set()
metadata_throughput = set()


## Create the elasticsearch connection

In [3]:
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'], timeout=6000)


## Generate the common part of the queries

In [7]:
# src_site_name = 'source_site_name'
# dest_site_name = 'destination_site_name'
# src_site_ip = 'source_site_ip'
# dest_site_ip = 'destination_site_ip'

timestamp = { 'gte': '2016-01-01', 'lt': '2016-06-01' }

my_query = {}
my_query['query'] = {}
my_query['query']['filtered'] = {}
my_query['query']['filtered']['query'] = { "match_all": {} }
my_query['query']['filtered']['filter'] = {}
my_query['query']['filtered']['filter']['bool'] = {}
my_query['query']['filtered']['filter']['bool']['must'] = []
# my_query['query']['filtered']['filter']['bool']['must'].append({ 'term': { 'srcSite': src_site_name } })
# my_query['query']['filtered']['filter']['bool']['must'].append({ 'term': { 'destSite': dest_site_name } })
my_query['query']['filtered']['filter']['bool']['must'].append({ 'range': { 'timestamp': timestamp } })
my_query['fielddata_fields'] = [ 'timestamp' ]

print(my_query)
print('===========================================')

my_index = "network_weather_2-*"

print(my_index)
print('===========================================')


{'query': {'filtered': {'query': {'match_all': {}}, 'filter': {'bool': {'must': [{'range': {'timestamp': {'lt': '2016-06-01', 'gte': '2016-01-01'}}}]}}}}, 'fielddata_fields': ['timestamp']}
network_weather_2-*


## 1/3 Collect the metadata of delay

In [8]:
my_type = 'latency'
scroll = list(helpers.scan(client=es, query=my_query, index=my_index, doc_type=my_type, request_timeout=6000))
count = 0
for result in scroll:
    count += 1
    info = result['_source']
    src_ip = info['src']
    dest_ip = info['dest']
    src_name = info['srcSite']
    dest_name = info['destSite']
    src_production = info['srcProduction']
    dest_production = info['destProduction']
    record = '{}\t{}\t{}\t{}\t{}\t{}'.format(src_ip, dest_ip, src_name, dest_name, src_production, dest_production)
    metadata_delay.add(record)
    if count % 1000 == 0:
        print('Count = {}\tSize = {}'.format(count, len(metadata_delay)))
    

KeyboardInterrupt: 